In [1]:
print('get qualified txs')
import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
import google_bq_utils as bqu
sys.path.pop()

import time

import numpy as np
import pandas as pd
import os

get qualified txs


In [2]:
trailing_days = 180 #Assume we always catch chains within 6 months

In [3]:
# Run Dune
print('     dune runs')
days_param = d.generate_query_parameter(input=trailing_days,field_name='trailing_days',dtype='number')
dune_df = d.get_dune_data(query_id = 3912454, #https://dune.com/queries/3912454
    name = "daily_op_stack_chains_l1_data",
    path = "outputs",
    performance="large",
    params = [days_param],
    num_hours_to_rerun=0 #always rereun
)
dune_df['source'] = 'dune'
dune_df['dt'] = pd.to_datetime(dune_df['dt']).dt.tz_localize(None)

2024-07-11 18:26:54.218 | INFO     | duneapi_utils:get_dune_data:58 - Results available at https://dune.com/queries/3912454?trailing_days=9999


     dune runs


2024-07-11 18:26:57,171 INFO dune_client.api.base executing 3912454 on large cluster
2024-07-11 18:26:57,487 INFO dune_client.api.base waiting for query execution 01J2HYG82Y5NEVKTXQW0S3GDKT to complete: ExecutionState.PENDING (queue position: 1)
2024-07-11 18:26:58,658 INFO dune_client.api.base waiting for query execution 01J2HYG82Y5NEVKTXQW0S3GDKT to complete: ExecutionState.PENDING (queue position: 1)
2024-07-11 18:26:59,823 INFO dune_client.api.base waiting for query execution 01J2HYG82Y5NEVKTXQW0S3GDKT to complete: ExecutionState.EXECUTING
2024-07-11 18:27:00,995 INFO dune_client.api.base waiting for query execution 01J2HYG82Y5NEVKTXQW0S3GDKT to complete: ExecutionState.EXECUTING
2024-07-11 18:27:02,165 INFO dune_client.api.base waiting for query execution 01J2HYG82Y5NEVKTXQW0S3GDKT to complete: ExecutionState.EXECUTING
2024-07-11 18:27:03,330 INFO dune_client.api.base waiting for query execution 01J2HYG82Y5NEVKTXQW0S3GDKT to complete: ExecutionState.EXECUTING
2024-07-11 18:27:04,5

In [6]:
dune_df.dtypes
dune_df['avg_l1_calldata_gas_price_on_l1_inbox'] = dune_df['avg_l1_calldata_gas_price_on_l1_inbox'].astype(float)

alignment_map                                    object
avg_blob_base_fee_on_l1_inbox                   float64
avg_l1_calldata_gas_price_on_l1_inbox           float64
avg_l1_gas_price_on_l1_inbox                    float64
avg_l1_gas_price_on_l1_output                   float64
calldata_bytes_l1_inbox                           int64
chain_id                                        float64
dt                                       datetime64[ns]
l1_blobgas_eth_fees_inbox                       float64
l1_blobgas_purchased_inbox                      float64
l1_blobgas_usd_fees_inbox                       float64
l1_calldata_eth_fees_inbox                      float64
l1_calldata_usd_fees_inbox                      float64
l1_eth_fees_combined                            float64
l1_eth_fees_inbox                               float64
l1_eth_fees_output                              float64
l1_gas_used_combined                              int64
l1_gas_used_inbox                               

In [7]:
#BQ Upload
bqu.append_and_upsert_df_to_bq_table(dune_df, 'daily_op_stack_chains_l1_data', unique_keys = ['chain_id','dt','name'])

Start Writing api_table_uploads.daily_op_stack_chains_l1_data_staging
Data loaded successfully to api_table_uploads.daily_op_stack_chains_l1_data_staging


2024-07-11 18:35:47,747 INFO google_bq_utils Append and upsert to api_table_uploads.daily_op_stack_chains_l1_data completed successfully
2024-07-11 18:35:47,956 INFO google_bq_utils Staging table oplabs-tools-data.api_table_uploads.daily_op_stack_chains_l1_data_staging deleted.
